In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import Counter

In [2]:
data = pd.read_csv("weather.csv")
data = data.drop(["Unnamed: 58"],axis=1)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
dct = {}
for sid in tqdm(set(data["M_SESSION_UID"])):
    data_ss = data[data["M_SESSION_UID"] ==sid]
    for ts in set(data_ss["TIMESTAMP"]):
        dct[(sid,ts)] = data_ss[data_ss["TIMESTAMP"] == ts]
Counter(list(map(lambda x: len(x), dct.values())))

Counter({77: 46393, 67: 1})

In [4]:
ls = []
sid_ts_table = {}
for (sid,ts),table in dct.items():
    num_samples = np.array(list(table["M_NUM_WEATHER_FORECAST_SAMPLES"]))
    assert np.all(num_samples[0] == num_samples)
    num_samples = num_samples[0]
    ls.append(num_samples)
    if num_samples!=0:
        sid_ts_table[(sid,ts)] = dct[(sid,ts)]
Counter(ls), len(sid_ts_table)

(Counter({3: 3014,
          10: 19141,
          0: 10743,
          27: 164,
          9: 3665,
          48: 8267,
          7: 1322,
          6: 78}),
 35651)

In [38]:
forecast_sample_cols = [
    'M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE',
    'M_WEATHER_FORECAST_SAMPLES_M_WEATHER',
    'M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE',
    'M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE',
    'M_TIME_OFFSET',
    'M_TRACK_TEMPERATURE_CHANGE',
    'M_AIR_TEMPERATURE_CHANGE',
    'M_RAIN_PERCENTAGE']

for (sid,ts),table in sid_ts_table.items():
    num_samples = list(table["M_NUM_WEATHER_FORECAST_SAMPLES"])[0]
    ses_type = table["M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE"]
    num_nans = ses_type.isna().sum()
    assert np.isnan(ses_type[:num_nans]).sum() == num_nans
    assert np.all(ses_type[num_nans+num_samples:] == 0)

In [35]:
sid_ts_filtered = {}
for (sid,ts),table in sid_ts_table.items():
    num_samples = list(table["M_NUM_WEATHER_FORECAST_SAMPLES"])[0]
    ses_type = table["M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE"]
    num_nans = ses_type.isna().sum()
    sid_ts_filtered[(sid,ts)] = table.iloc[num_nans:num_nans+num_samples]
lens = [len(el) for el in sid_ts_filtered.values()]
Counter(lens), sum(lens)

(Counter({3: 3014, 10: 19141, 27: 164, 9: 3665, 48: 8267, 7: 1322, 6: 78}),
 644403)

In [82]:
sid_ts_filtered_cols_dropped = {k:v[forecast_sample_cols+["M_TRACK_ID"]] for k,v in sid_ts_filtered.items()}

In [84]:
sid_ts_filtered_cols_dropped[list(sid_ts_filtered_cols_dropped.keys())[0]]

,M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE,M_WEATHER_FORECAST_SAMPLES_M_WEATHER,M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE,M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE,M_TIME_OFFSET,M_TRACK_TEMPERATURE_CHANGE,M_AIR_TEMPERATURE_CHANGE,M_RAIN_PERCENTAGE,M_TRACK_ID
114751,10.0,0.0,31.0,25.0,0.0,2.0,2.0,2.0,3
114752,10.0,0.0,31.0,25.0,5.0,2.0,2.0,2.0,3
114753,10.0,0.0,31.0,25.0,10.0,2.0,2.0,1.0,3


In [86]:
final_df = {}
for (sid,ts),table in sid_ts_filtered_cols_dropped.items():
    for sess_type in set(table["M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE"]):
        final_df[(sid,ts,sess_type)] = table[table["M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE"]==sess_type]
 

In [87]:
sum([len(el)-1 for el in final_df.values()])

539439

In [88]:
len(final_df)

104964